In [ ]:
import pandas as pd
import numpy as np
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from lofo import LOFOImportance, plot_importance, Dataset
import lightgbm as lgbm
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import optuna
from IPython.core.display import HTML
import dataframe_image as dfi
from sklearn.metrics import mutual_info_score
import seaborn as sns
# pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
train_in = pd.read_csv('C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/data/application_train.csv')
test_in = pd.read_csv('C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/data/application_test.csv')
print('Train set shape: ', train_in.shape)
print('Test set shape: ', test_in.shape)
train_in.columns = train_in.columns.str.lower()
test_in.columns = test_in.columns.str.lower()

In [ ]:
print(train_in.columns)
print([x for x in train_in.columns if x not in test_in.columns])
d_types = train_in.dtypes

In [ ]:
# inspections.score.describe()
# def test_dependency(df, col1, col2):
#     '''Tests whether there is total redundancy betwen col1 and col2
#        by testing the number of unique combinations'''
#     unique_1 = len(df[col1].unique())
#     unique_2 = len(df[col2].unique())
#     unique_combos = len(df[[col1, col2]].drop_duplicates())
#     if unique_1 == 2 and unique_2 == 2:
#         if list(sorted(df[col1].unique())) == [0, 1] and unique_combos == max(unique_1, unique_2):
#             print("Unique combos of {} and {}: {}".format(col1, col2, unique_combos))

# for col1 in train_in.columns:
#     for col2 in train_in.columns:
#         if col2 == col1 or col1 in ['sk_id_curr', 'target'] or col2 in ['sk_id_curr', 'target']:
#             continue
#         test_dependency(train_in, col1, col2)

In [ ]:
unique_cols = [i for i in train_in.columns if i not in test_in.columns]
unique_cols

In [ ]:
# Test if these variables are a unique identifier
possible_identifier = ['sk_id_curr']
print("# of duplicates: ",sum(train_in.duplicated(possible_identifier)))

In [ ]:
for col in train_in.columns:
    if len(train_in[col].unique()) > 10:
        temp = train_in.copy()
        temp['count'] = temp.groupby(col)['sk_id_curr'].transform('count')
        temp['percentage'] = temp['count'] / len(temp)
        if temp['percentage'].max() > 0.15:
            print('{}: {}, {:,}, {:.1%}'.format(col, temp[temp['percentage'] > 0.15].reset_index().loc[0][col], temp[temp['percentage'] > 0.15].reset_index().loc[0]['count'], temp[temp['percentage'] > 0.15].reset_index().loc[0]['percentage']))

In [ ]:
for col in train_in.columns:
    if 'XNA' in train_in[col].unique():
        print(col)

In [ ]:
def compute_profile_metrics(data):
    
    metrics = pd.DataFrame()
    
    for col, index in zip(data.columns, range(0, len(data.columns))):
        if col == 'target':
            continue
        
        if col == 'days_employed':
            data[col] = np.where((data[col] == 365243), np.nan, data[col])
        
        if isinstance(data.reset_index().loc[0][col], str) or sorted(list(data[col].unique())) == [0, 1]:
            data[col] = np.where((data[col].isnull() == True) | (data[col] == 'XNA'), '', data[col])
            
            missing_perc = len(data[(data[col].isnull() == True) | (data[col] == '')]) / len(data)
            
            data[col] = np.where((data[col] == 'Yes'), 'Y', data[col])
            data[col] = np.where((data[col] == 'No'), 'N', data[col])
            
            if sorted(list(data[col].unique())) == ['0', '1'] or sorted(list(data[col].unique())) == ['', '0', '1']:
                data[col] = data[col].astype(float)
            
            dtype = data.dtypes[index].name
            num_vals = len(list(data[col].unique()))
            
            if sorted(list(data[col].unique())) == [0, 1] or sorted(list(data[col].unique())) == [0, 1, np.nan] or sorted(list(data[col].unique())) == ['N', 'Y'] or sorted(list(data[col].unique())) == ['', 'N', 'Y']:
                perc_yes = len(data[(data[col] == 'Y') | (data[col] == 1)]) / len(data)
                metrics = metrics.append({'column': col, 'dtype': dtype, 'perc_yes': perc_yes, 'unique_vals': num_vals, 'missing_perc': missing_perc}, ignore_index=True)
            else:
                metrics = metrics.append({'column': col, 'dtype': dtype, 'perc_yes': np.nan, 'unique_vals': num_vals, 'missing_perc': missing_perc}, ignore_index=True)
        else:
            median = data[col].median()
            mean = data[col].mean()
            min_val = data[col].min()
            max_val = data[col].max()
            dtype = data.dtypes[index].name
            num_vals = len(list(data[col].unique()))
            missing_perc = len(data[(data[col].isnull() == True) | (data[col] == '')]) / len(data)
            metrics = metrics.append({'column': col, 'dtype': dtype, 'median': median, 'mean': mean, 'min': min_val, 'max': max_val, 'unique_vals': num_vals, 'missing_perc': missing_perc}, ignore_index=True)
    
    return metrics, data

In [ ]:
train = train_in.copy()
metrics, train = compute_profile_metrics(train)

In [ ]:
ads_used = ['EXT_SOURCE_3',
             'EXT_SOURCE_2',
             'EXT_SOURCE_1',
             'NAME_EDUCATION_TYPE',
             'CODE_GENDER',
             'NAME_INCOME_TYPE',
             'ORGANIZATION_TYPE',
             'DAYS_EMPLOYED',
             'FLOORSMAX_AVG',
             'FLOORSMAX_MEDI',
             'FLOORSMAX_MODE',
             'EMERGENCYSTATE_MODE',
             'HOUSETYPE_MODE',
             'AMT_GOODS_PRICE',
             'REGION_POPULATION_RELATIVE',
             'OWN_CAR_AGE',
             'DAYS_REGISTRATION',
             'OCCUPATION_TYPE',
             'FLAG_DOCUMENT_3',
             'REG_CITY_NOT_LIVE_CITY',
             'FLAG_EMP_PHONE',
             'REG_CITY_NOT_WORK_CITY',
             'DAYS_ID_PUBLISH',
             'DAYS_LAST_PHONE_CHANGE',
             'REGION_RATING_CLIENT',
             'REGION_RATING_CLIENT_W_CITY',
             'DAYS_BIRTH']

ads_used_lower = [x.lower() for x in ads_used]
pd.options.display.max_rows = 150
metrics_all = metrics.copy()
metrics_used = metrics.copy()
for df, file_name in zip([metrics_all, metrics_used], ['metrics_all_profiled', 'metrics_used_profiled']):
    if 'used' in file_name:
        df = df[df['column'].isin(ads_used_lower)]
    for col in ['missing_perc', 'perc_yes']:
        df[col] = df[col].apply('{:.1%}'.format)
        df[col] = np.where((df[col] == 'nan%'), '', df[col])
    for col in ['median', 'mean', 'min', 'max']:
        df[col] = df[col].apply('{:,.1f}'.format)
    for col in ['unique_vals']:
        df[col] = df[col].astype(int)
        df[col] = df[col].apply('{:,}'.format)
    if 'all' in file_name:
        temp1 = df.copy()
        temp2 = df.copy()
        temp3 = df.copy()
        temp1 = temp1.loc[0:40]
        temp2 = temp2.loc[41:80]
        temp3 = temp3.loc[81:]
        for temp, file_name_temp in zip([temp1, temp2, temp3], ['_1', '_2', '_3']):
            dfi.export(temp.style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/{}{}.png'.format(file_name, file_name_temp), max_rows=150)
    else:
        dfi.export(df.style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/{}.png'.format(file_name), max_rows=150)
    if 'used' in file_name:
        display(df)

In [ ]:
def plot_dist(df_in, col_list):
    
    df = df_in.copy()
    df = df[col_list]
    
    for col in df:
        plt.figure(figsize=(20,10))
        if 'days' in col:
            df[col] = df[col] / -365
            bins = np.arange(0, df[col].max(), 1)
            plt.hist(df[col], bins=bins)
            plt.xticks(bins)
            title = col.title() + ' In Years'
        else:
            plt.hist(df[col])
            title = col.title()
        plt.title('Distribution of {}'.format(title))
        plt.xlabel(title.lower())
        plt.ylabel('count')
        if len(df[col].unique()) > 4:
            plt.xticks(rotation=45)
        plt.savefig('C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/{}_hist.png'.format(col))
        plt.show()
        
        
plot_dist(train, ads_used_lower)

In [ ]:
temp = train.copy()
    
cat_vars = list(metrics[(metrics['dtype'] == 'object') & ((metrics['unique_vals'] > 3) | ((metrics['unique_vals'] == 3) & (metrics['missing_perc'] == 0)))]['column'])

for col in temp.columns:
    if col in list(metrics[(metrics['dtype'] == 'object')]['column']) and col not in cat_vars:
        unique_vals = list(temp[col].unique())
        print('{} with {} coded as 0'.format(col, unique_vals[0]))
        temp[col] = np.where((temp[col] == unique_vals[0]), 0, 1)
        
seen = []
corrs = pd.DataFrame()
for col1 in temp.columns:
    if col1 == 'target' or col1[-4:] == 'mode' or col1[-4:] == 'medi' or col1[-4:] == '_avg' or 'flag_document' in col1 or col1 in cat_vars:
        continue
    for col2 in temp.columns:
        if col1 == col2 or col2 + ', ' + col1 in seen or col2 in cat_vars or col2 == 'target':
            continue
        temp1 = temp.copy()
        temp1 = temp1[(temp1[col1].isnull() == False) & (temp1[col1] != '') & (temp1[col2].isnull() == False) & (temp1[col2] != '')]
        corr = np.corrcoef(temp1[col1], temp1[col2])[0,1]
        seen.append(col1 + ', ' + col2)
        corrs = corrs.append({'col1': col1, 'col2': col2, 'corr': corr}, ignore_index=True)

corrs_target = pd.DataFrame()
for col in temp.columns:
    if col not in cat_vars and col != 'target':
        temp1 = temp.copy()
        temp1 = temp1[(temp1[col].isnull() == False) & (temp1[col] != '')]
        corr = np.corrcoef(temp1['target'], temp1[col])[0,1]
        corrs_target = corrs_target.append({'col': col, 'corr': corr}, ignore_index=True)

In [ ]:
corrs['abs_corr'] = abs(corrs['corr'])
corrs_target['abs_corr'] = abs(corrs_target['corr'])
dfi.export(corrs[abs(corrs['corr']) > 0.70].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/func_dependencies.png', max_rows=150)
dfi.export(corrs.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_nontarget.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[0:20].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_1.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[21:40].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_2.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[41:60].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_3.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[61:80].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_4.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[81:100].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_5.png', max_rows=150)
dfi.export(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[101:].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_nonfunctional_nontarget_6.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[0:20].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_1.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[21:40].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_2.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[41:60].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_3.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[61:80].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_4.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[81:100].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_5.png', max_rows=150)
dfi.export(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).reset_index(drop=True).loc[101:].style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_all_target_6.png', max_rows=150)
dfi.export(corrs_target[(abs(corrs_target['corr']) >= 0.03)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1).style.hide_index().set_table_styles([dict(selector = 'th', props=[('text-align', 'right')])]), 'C:/Users/bbloc/OneDrive/Desktop/NYU_Data_Science/Responsible_DS/Project/figures/correlations_large_target.png', max_rows=150)
display(corrs[abs(corrs['corr']) > 0.70])
display(corrs[(abs(corrs['corr']) > 0.15) & (abs(corrs['corr']) <= 0.70)].sort_values(by='abs_corr', ascending=False).drop(['abs_corr'], axis=1))
display(corrs_target.sort_values(by='abs_corr', ascending=False).drop(['abs_corr'],axis=1))
correlations = train.corr()['target'].sort_values()
correlations

In [ ]:
train['age_years'] = np.ceil((train['days_birth'] * -1) / 365)
train['employed_years'] = np.ceil((train['days_employed'] * -1) / 365)
for col in ['code_gender', 'name_education_type', 'name_family_status', 'name_housing_type', 'age_years', 'employed_years']:
    print(train['amt_income_total'].median())
    display(train.groupby(col)['amt_income_total'].median())

In [ ]:
# From this point on, we will implement Hanane's final model and examine the outcomes

df = train_in.copy()

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

label_encode = LabelEncoder()

for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            label_encode.fit(df[col])
            df[col] = label_encode.transform(df[col])
            

df = pd.get_dummies(df)
display(df.head(1))

ft_list = ['EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'EXT_SOURCE_1',
 'NAME_EDUCATION_TYPE_Higher education',
 'CODE_GENDER_F',
 'NAME_INCOME_TYPE_Pensioner',
 'ORGANIZATION_TYPE_XNA',
 'DAYS_EMPLOYED',
 'FLOORSMAX_AVG',
 'FLOORSMAX_MEDI',
 'FLOORSMAX_MODE',
 'EMERGENCYSTATE_MODE_NO',
 'HOUSETYPE_MODE_block of flats',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'OWN_CAR_AGE',
 'DAYS_REGISTRATION',
 'OCCUPATION_TYPE_Laborers',
 'FLAG_DOCUMENT_3',
 'REG_CITY_NOT_LIVE_CITY',
 'FLAG_EMP_PHONE',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'REG_CITY_NOT_WORK_CITY',
 'DAYS_ID_PUBLISH',
 'CODE_GENDER_M',
 'DAYS_LAST_PHONE_CHANGE',
 'NAME_INCOME_TYPE_Working',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'DAYS_BIRTH',
 'TARGET']

ft_list = [x.lower() for x in ft_list]
df = df[ft_list]

# create X_train, y_train
X_train = df.drop('target', axis = 1)
y_train = df['target']
target = y_train
features = X_train
# Feature names
features_list = list(X_train.columns)

#Impute missing values using median strategy
imputer = SimpleImputer(strategy = 'median')

#Scale all values in 0-1 range
scaler = MinMaxScaler(feature_range = (0, 1))

#fit the imputer on training set
imputer.fit(X_train)

#transform both training and testing data
X_train = imputer.transform(X_train)

#fit scaler and transform
scaler.fit(X_train)
train_scaled = scaler.transform(X_train)


In [ ]:
tuned_lgbm_model = lgbm.LGBMClassifier(n_estimators= 500, learning_rate = 0.09913706634323802, 
                                       num_leaves = 900, max_depth = 8, min_data_in_leaf = 600)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
tuned_lgbm_model.fit(train_features, train_labels)

In [ ]:
tuned_lgbm_predictions = tuned_lgbm_model.predict(test_features)

In [ ]:
def get_scores (preds): 
    print('Accuracy: %.3f' % accuracy_score(test_labels.values, preds))
    print('Precision: %.3f' % precision_score(test_labels.values, preds))
    print('Recall: %.3f' % recall_score(test_labels.values, preds))
    print('F1: %.3f' % f1_score(test_labels.values, preds))

print('Tuned LGBM scores: ')
get_scores(tuned_lgbm_predictions)

In [ ]:
print(len(tuned_lgbm_predictions))
print(np.sum(tuned_lgbm_predictions))
print(np.sum(test_labels))

In [ ]:
df_test = pd.DataFrame(test_features, columns=[x for x in df.columns if x != 'target'])
df_test['true'] = list(test_labels)
df_test['pred'] = tuned_lgbm_predictions

df_test['age_years'] = np.ceil((df_test['days_birth'] * -1) / 365)
df_test['emp_years'] = np.ceil((df_test['days_employed'] * -1) / 365)

df_test['age_group'] = ''
df_test['age_group'] = np.where((df_test['age_years'] < 30), '<30', df_test['age_group'])
df_test['age_group'] = np.where((df_test['age_years'] >= 30) & (df_test['age_years'] < 40), '30-39', df_test['age_group'])
df_test['age_group'] = np.where((df_test['age_years'] >= 40) & (df_test['age_years'] < 50), '40-49', df_test['age_group'])
df_test['age_group'] = np.where((df_test['age_years'] >= 50) & (df_test['age_years'] < 60), '50-59', df_test['age_group'])
df_test['age_group'] = np.where((df_test['age_years'] >= 60), '60+', df_test['age_group'])

df_test['emp_group'] = ''
df_test['emp_group'] = np.where((df_test['emp_years'] >= 0) & (df_test['emp_years'] < 5), '0-4', df_test['emp_group'])
df_test['emp_group'] = np.where((df_test['emp_years'] >= 5) & (df_test['emp_years'] < 10), '5-9', df_test['emp_group'])
df_test['emp_group'] = np.where((df_test['emp_years'] >= 10) & (df_test['emp_years'] < 20), '10-19', df_test['emp_group'])
df_test['emp_group'] = np.where((df_test['emp_years'] >= 20), '20+', df_test['emp_group'])

In [ ]:
def compute_metrics(df, var):

    table = pd.DataFrame()
    
    for sub_pop in df[var].unique():

        df_filt = df.copy()
        df_filt = df_filt[df_filt[var] == sub_pop]
        fp = 0
        tp = 0
        fn = 0
        tn = 0
        for index, row in df_filt.iterrows():
            if row['pred'] == 1 and row['true'] == 0:
                fp += 1
            elif row['pred'] == 0 and row['true'] == 1:
                fn += 1
            elif row['pred'] == 1 and row['true'] == 1:
                tp += 1
            elif row['pred'] == 0 and row['true'] == 0:
                tn += 1
                
        accuracy = (tp + tn) / len(df_filt)
        fp_rate = fp / (fp + tn)
        fn_rate = 1 - (fn / (tp + fn))
        
        table = table.append({'group': var + ' ' + str(sub_pop), 'acc': accuracy, 'fp': fp_rate, 'fn': fn_rate}, ignore_index=True)

#         print("Accuracy for {} {}: {:.1%}".format(var, sub_pop, accuracy))
#         print("FP Rate for {} {}: {:.1%}".format(var, sub_pop, fp_rate))
#         print("FN Rate for {} {}: {:.1%}".format(var, sub_pop, fn_rate))
#         print("\n")
    display(table)


In [ ]:
compute_metrics(df_test, 'code_gender_f')
compute_metrics(df_test, 'age_group')
compute_metrics(df_test, 'emp_group')

In [ ]:
temp = train.copy()
temp = temp[(temp['floorsmax_avg'].isnull() == False) & (temp['floorsmax_avg'] != '') & (temp['floorsmax_medi'].isnull() == False) & (temp['floorsmax_medi'] != '')]
corr = np.corrcoef(temp['floorsmax_avg'], temp['floorsmax_medi'])[0,1]
print(corr)

In [ ]:
from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics import BinaryLabelDatasetMetric

In [ ]:
# convert this dataframe into an aif360 dataset
df_nonull = df.copy()
for col in df_nonull.columns:
    df_nonull = df_nonull[(df_nonull[col].isnull() == False) & (df_nonull[col] != '')]
protected_attr = 'code_gender_f'
target = 'target'
dataset_orig = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=df_nonull,
    label_names=[target],
    protected_attribute_names=[protected_attr])
privileged_groups = [{protected_attr: 0}] 
unprivileged_groups = [{protected_attr: 1}]

In [ ]:
# Convert to dataframes
train_orig, test_orig = dataset_orig.split([0.8], shuffle=True, seed=10)
train_orig, val_orig = train_orig.split([0.875], shuffle=True, seed=5)

# Convert to dataframes
train_orig_df, _ = train_orig.convert_to_dataframe()
val_orig_df, _ = val_orig.convert_to_dataframe()
test_orig_df, _ = test_orig.convert_to_dataframe()

print("Train set: ", train_orig_df.shape)
print("Val set: ", val_orig_df.shape)
print("Test set: ", test_orig_df.shape)

In [ ]:
lgbm_aif = lgbm.LGBMClassifier(n_estimators= 500, learning_rate = 0.09913706634323802, 
                                       num_leaves = 900, max_depth = 8, min_data_in_leaf = 600)

X_train_aif = train_orig_df.drop(['target'],axis=1).values
y_train_aif = train_orig_df[['target']].values
X_test_aif = test_orig_df.drop(['target'],axis=1).values
y_test_aif = test_orig_df[['target']].values

In [ ]:
lgbm_aif.fit(X_train_aif, y_train_aif)
y_pred = lgbm_aif.predict(X_test_aif)

In [ ]:
def compute_acc(y_pred, y_test):

    correct = 0

    for pred, true in zip(y_pred, y_test):
        if pred == true:
            correct += 1

    return correct / len(y_pred)

def compute_acc_group(y_pred, y_test, test_orig_df):

    test_orig_df = test_orig_df.reset_index(drop=True)

    correct_priv = 0
    correct_unpriv = 0

    total_priv = len(test_orig_df[test_orig_df['code_gender_f'] == 0])
    total_unpriv = len(test_orig_df[test_orig_df['code_gender_f'] == 1])

    for index, row in test_orig_df.iterrows():
        if row['code_gender_f'] == 0 and y_pred[index] == y_test[index][0]:
            correct_priv += 1
        elif row['code_gender_f'] == 1 and y_pred[index] == y_test[index][0]:
            correct_unpriv += 1
    
    return correct_priv / total_priv, correct_unpriv / total_unpriv

def compute_fp(y_pred, y_test, test_orig_df):

    fp_priv = 0
    tn_priv = 0
    fp_unpriv = 0
    tn_unpriv = 0

    test_orig_df = test_orig_df.reset_index(drop=True)

    for index, row in test_orig_df.iterrows():
        if row['code_gender_f'] == 0 and y_pred[index] == 1 and y_test[index][0] == 0:
            fp_priv += 1
        elif row['code_gender_f'] == 0 and y_pred[index] == 0 and y_test[index][0] == 0:
            tn_priv += 1
        elif row['code_gender_f'] == 1 and y_pred[index] == 1 and y_test[index][0] == 0:
            fp_unpriv += 1
        elif row['code_gender_f'] == 1 and y_pred[index] == 0 and y_test[index][0] == 0:
            tn_unpriv += 1

    fp_rate_priv = fp_priv / (fp_priv + tn_priv)
    fp_rate_unpriv = fp_unpriv / (fp_unpriv + tn_unpriv)

    return fp_rate_priv, fp_rate_unpriv

def calc_disparate_impact(df, y_pred):

    df_pred = df.copy()
    df_pred['target'] = y_pred

    privileged_groups = [{'code_gender_f': 0}]
    unprivileged_groups = [{'code_gender_f': 1}]  

    aif360_test = StandardDataset(df_pred, label_name='target', protected_attribute_names=['code_gender_f'], 
                privileged_classes=[[1]], favorable_classes=[1])
  
    metric_preds = BinaryLabelDatasetMetric(
    aif360_test, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )

    disparate_impact = metric_preds.disparate_impact()

    return disparate_impact

def calc_fp_diff(df, y_pred):

    privileged_groups = [{'code_gender_f': 0}]
    unprivileged_groups = [{'code_gender_f': 1}]

    orig_aif360 = StandardDataset(df, label_name='target', protected_attribute_names=['code_gender_f'], 
                privileged_classes=[[1]], favorable_classes=[1])
    pred_test_df = df.copy()
    pred_test_df['target'] = y_pred
    preds_aif360 = StandardDataset(pred_test_df, label_name='target', protected_attribute_names=['code_gender_f'], 
                  privileged_classes=[[1]], favorable_classes=[1])

    orig_vs_preds_metrics = ClassificationMetric(orig_aif360, preds_aif360,
                                                    unprivileged_groups=unprivileged_groups,
                                                    privileged_groups=privileged_groups)

    fp_rate_priv = orig_vs_preds_metrics.false_positive_rate(privileged=True)
    fp_rate_unpriv = orig_vs_preds_metrics.false_positive_rate(privileged=False)

    fp_diff = fp_rate_unpriv - fp_rate_priv

    return fp_diff

accuracy = compute_acc(y_pred, y_test_aif)
accuracy_priv, accuracy_unpriv = compute_acc_group(y_pred, y_test_aif, test_orig_df)
disparate_impact = calc_disparate_impact(test_orig_df, y_pred)
fp_diff = calc_fp_diff(test_orig_df, y_pred)

print("Accuracy of the model: {:.3%}".format(accuracy))
print("Accuracy of the privelaged group: {:.3%}".format(accuracy_priv))
print("Accuracy of the unprivelaged group: {:.3%}".format(accuracy_unpriv))
print("Disparate Impact: {:.3}".format(disparate_impact))
print("False Positive Rate Difference: {:.3%}".format(fp_diff))

In [ ]:
train[train['code_gender'] == 'M']['target'].value_counts().plot(kind='bar')
plt.title('Distribution of Loan Default for Males')
plt.xlabel("Credit (1 = Default, 0 = Default)")
plt.ylabel("Frequency")
plt.show()

train[train['code_gender'] == 'F']['target'].value_counts().plot(kind='bar')
plt.title('Distribution of Loan Default for Females')
plt.xlabel("(1 = Default, 0 = No Default)")
plt.ylabel("Frequency")
plt.show()

print(len(train[(train['code_gender'] == 'M') & (train['target'] == 1)]) / len(train[(train['code_gender'] == 'M')]))
print(len(train[(train['code_gender'] == 'F') & (train['target'] == 1)]) / len(train[(train['code_gender'] == 'F')]))

print(len(train[(train['code_gender'] == 'M') & (train['amt_income_total'].isnull() == True)]) / len(train[(train['code_gender'] == 'M')]))
print(len(train[(train['code_gender'] == 'F') & (train['amt_income_total'].isnull() == True)]) / len(train[(train['code_gender'] == 'F')]))

In [ ]:
# convert this dataframe into an aif360 dataset
df_nonull = df.copy()
df_nonull['age_years'] = np.ceil((df_nonull['days_birth'] * -1) / 365)
df_nonull['age_group'] = np.where(df_nonull['age_years'] < 40, 1, 0)
for col in df_nonull.columns:
    df_nonull = df_nonull[(df_nonull[col].isnull() == False) & (df_nonull[col] != '')]
protected_attr = 'age_group'
target = 'target'
dataset_orig = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=df_nonull,
    label_names=[target],
    protected_attribute_names=[protected_attr])
privileged_groups = [{protected_attr: 0}] 
unprivileged_groups = [{protected_attr: 1}]

In [ ]:
# Convert to dataframes
train_orig, test_orig = dataset_orig.split([0.8], shuffle=True, seed=10)
train_orig, val_orig = train_orig.split([0.875], shuffle=True, seed=5)

# Convert to dataframes
train_orig_df, _ = train_orig.convert_to_dataframe()
val_orig_df, _ = val_orig.convert_to_dataframe()
test_orig_df, _ = test_orig.convert_to_dataframe()

print("Train set: ", train_orig_df.shape)
print("Val set: ", val_orig_df.shape)
print("Test set: ", test_orig_df.shape)

In [ ]:
lgbm_aif = lgbm.LGBMClassifier(n_estimators= 500, learning_rate = 0.09913706634323802, 
                                       num_leaves = 900, max_depth = 8, min_data_in_leaf = 600)

X_train_aif = train_orig_df.drop(['target'],axis=1).values
y_train_aif = train_orig_df[['target']].values
X_test_aif = test_orig_df.drop(['target'],axis=1).values
y_test_aif = test_orig_df[['target']].values

In [ ]:
lgbm_aif.fit(X_train_aif, y_train_aif)
y_pred = lgbm_aif.predict(X_test_aif)

In [ ]:
def compute_acc(y_pred, y_test):

    correct = 0

    for pred, true in zip(y_pred, y_test):
        if pred == true:
            correct += 1

    return correct / len(y_pred)

def compute_acc_group(y_pred, y_test, test_orig_df):

    test_orig_df = test_orig_df.reset_index(drop=True)

    correct_priv = 0
    correct_unpriv = 0

    total_priv = len(test_orig_df[test_orig_df['age_group'] == 0])
    total_unpriv = len(test_orig_df[test_orig_df['age_group'] == 1])

    for index, row in test_orig_df.iterrows():
        if row['age_group'] == 0 and y_pred[index] == y_test[index][0]:
            correct_priv += 1
        elif row['age_group'] == 1 and y_pred[index] == y_test[index][0]:
            correct_unpriv += 1
    
    return correct_priv / total_priv, correct_unpriv / total_unpriv

def compute_fp(y_pred, y_test, test_orig_df):

    fp_priv = 0
    tn_priv = 0
    fp_unpriv = 0
    tn_unpriv = 0

    test_orig_df = test_orig_df.reset_index(drop=True)

    for index, row in test_orig_df.iterrows():
        if row['age_group'] == 0 and y_pred[index] == 1 and y_test[index][0] == 0:
            fp_priv += 1
        elif row['age_group'] == 0 and y_pred[index] == 0 and y_test[index][0] == 0:
            tn_priv += 1
        elif row['age_group'] == 1 and y_pred[index] == 1 and y_test[index][0] == 0:
            fp_unpriv += 1
        elif row['age_group'] == 1 and y_pred[index] == 0 and y_test[index][0] == 0:
            tn_unpriv += 1

    fp_rate_priv = fp_priv / (fp_priv + tn_priv)
    fp_rate_unpriv = fp_unpriv / (fp_unpriv + tn_unpriv)

    return fp_rate_priv, fp_rate_unpriv

def calc_disparate_impact(df, y_pred):

    df_pred = df.copy()
    df_pred['target'] = y_pred

    privileged_groups = [{'age_group': 0}]
    unprivileged_groups = [{'age_group': 1}]  

    aif360_test = StandardDataset(df_pred, label_name='target', protected_attribute_names=['age_group'], 
                privileged_classes=[[1]], favorable_classes=[1])
  
    metric_preds = BinaryLabelDatasetMetric(
    aif360_test, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
  )

    disparate_impact = metric_preds.disparate_impact()

    return disparate_impact

def calc_fp_diff(df, y_pred):

    privileged_groups = [{'age_group': 0}]
    unprivileged_groups = [{'age_group': 1}]

    orig_aif360 = StandardDataset(df, label_name='target', protected_attribute_names=['age_group'], 
                privileged_classes=[[1]], favorable_classes=[1])
    pred_test_df = df.copy()
    pred_test_df['target'] = y_pred
    preds_aif360 = StandardDataset(pred_test_df, label_name='target', protected_attribute_names=['age_group'], 
                  privileged_classes=[[1]], favorable_classes=[1])

    orig_vs_preds_metrics = ClassificationMetric(orig_aif360, preds_aif360,
                                                    unprivileged_groups=unprivileged_groups,
                                                    privileged_groups=privileged_groups)

    fp_rate_priv = orig_vs_preds_metrics.false_positive_rate(privileged=True)
    fp_rate_unpriv = orig_vs_preds_metrics.false_positive_rate(privileged=False)

    fp_diff = fp_rate_unpriv - fp_rate_priv

    return fp_diff

accuracy = compute_acc(y_pred, y_test_aif)
accuracy_priv, accuracy_unpriv = compute_acc_group(y_pred, y_test_aif, test_orig_df)
disparate_impact = calc_disparate_impact(test_orig_df, y_pred)
fp_diff = calc_fp_diff(test_orig_df, y_pred)

print("Accuracy of the model: {:.3%}".format(accuracy))
print("Accuracy of the privelaged group: {:.3%}".format(accuracy_priv))
print("Accuracy of the unprivelaged group: {:.3%}".format(accuracy_unpriv))
print("Disparate Impact: {:.3}".format(disparate_impact))
print("False Positive Rate Difference: {:.3%}".format(fp_diff))